In [ ]:
from datasets import load_dataset

# Load only the test split from the dataset
test_data = load_dataset("cfilt/iitb-english-hindi", split="test")

# You now have the test data
print(test_data)





import torch
#from datasets import load_dataset
from tokenizers import tokenizers
from tokenizers.models import WordLevel
from torch.utils.data import Dataset, DataLoader, random_split

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
import random
random.seed(42)
def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

config = get_config()

def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer


# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, test_data, 'en')
tokenizer_tgt = get_or_build_tokenizer(config, test_data, 'hi')



import torch
import torch.nn as nn
from torch.utils.data import Dataset

class EnToHinDataset(Dataset):

    def __init__(self, ds, tk_src, tk_tgt, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tk_src = tk_src
        self.tk_tgt = tk_tgt

        # Special tokens
        self.sos = torch.tensor([tk_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos = torch.tensor([tk_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad = torch.tensor([tk_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        # Get the source and target text from the dataset
        dic = self.ds[idx]
        src_text = dic['translation']['en']
        tgt_text = dic['translation']['hi']

        # Tokenize the source and target text
        en = self.tk_src.encode(src_text).ids
        de = self.tk_tgt.encode(tgt_text).ids

        # Calculate the number of padding tokens needed
        enc_pad_len = self.seq_len - len(en) - 2  # for <sos> and <eos>
        dec_pad_len = self.seq_len - len(de) - 1  # only <sos> at the beginning

        # Check if the sentence is too long
        if enc_pad_len < 0 or dec_pad_len < 0:
            raise ValueError("Sentence is too long")

        # Create the encoder input by adding <sos>, <eos>, and padding
        en_inp = torch.cat([
            self.sos,
            torch.tensor(en, dtype=torch.int64),
            self.eos,
            torch.tensor([self.pad] * enc_pad_len, dtype=torch.int64)
        ])

        # Create the decoder input by adding <sos> and padding
        de_inp = torch.cat([
            self.sos,
            torch.tensor(de, dtype=torch.int64),
            torch.tensor([self.pad] * dec_pad_len, dtype=torch.int64)
        ])

        # Create the label by adding <eos> at the end and padding
        label = torch.cat([
            torch.tensor(de, dtype=torch.int64),
            self.eos,
            torch.tensor([self.pad] * dec_pad_len, dtype=torch.int64)
        ])

        # Return a dictionary containing the inputs and labels
        return {
            "encoder_input": en_inp,  # Encoder input
            "decoder_input": de_inp,  # Decoder input
            "label": label,  # Target labels
            "encoder_mask": (en_inp != self.pad).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (de_inp != self.pad).unsqueeze(0).int() & causal_mask(de_inp.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "src_text": src_text,
            "tgt_text": tgt_text,
        }


def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

causal_mask(3)



from torch.utils.data import Dataset, DataLoader, random_split
# Train and test splits
data = test_data
train_ds_size = int(0.9 * len(data))
val_ds_size = len(data) - train_ds_size
train_data, val_data = random_split(data, [train_ds_size, val_ds_size])




max_len_src = 0
max_len_tgt = 0
for item in test_data:
  src_ids = tokenizer_src.encode(item['translation']['en']).ids
  tgt_ids = tokenizer_tgt.encode(item['translation']['hi']).ids
  max_len_src = max(max_len_src, len(src_ids))
  max_len_tgt = max(max_len_tgt, len(tgt_ids))
print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')




seq_len = 100

t_ds = EnToHinDataset(train_data, tokenizer_src, tokenizer_tgt, 100)
v_ds = EnToHinDataset(val_data, tokenizer_src, tokenizer_tgt, 100)


from torch.utils.data import DataLoader

t_dl = DataLoader(t_ds, batch_size=16, shuffle=True)
v_dl = DataLoader(v_ds, batch_size=16, shuffle=True)


for i in t_dl:
  print(i['encoder_input'].shape)
  print(i['decoder_input'].shape)
  print(i['label'].shape)
  print(i['encoder_mask'].shape)
  print(i['decoder_mask'].shape)
  break




src_vocab_size = tokenizer_src.get_vocab_size()
tgt_vocab_size = tokenizer_tgt.get_vocab_size()






import torch
import torch.nn as nn
from torch.nn import functional as F

vocab_size = 100
batch_size = 16
block_size = 100  # seq_len
n_embd = 50
n_head = 2
n_layer = 2
dropout = 0.2
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_iters = 10000 # max no of steps for training
eval_interval = 100 # after how many steps the evaluation will take place
eval_iters = 1000  # how many sample of batches will use for evaluation

lr = 3e-4




class DecoderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, y ,z, mask):


        mask = mask.squeeze(1)

        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)



        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)
        att = att.masked_fill(mask == 0, float('-inf'))
        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)


        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)
        return out

class EnocderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, mask=None):

        mask = mask.squeeze(1)

        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)



        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)

        att = att.masked_fill(mask == 0, float('-inf'))

        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)

        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)

        return out

class EncoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([EnocderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x, mask):
        out = torch.cat([h(x, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class DecoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([DecoderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y ,z, mask):
        out = torch.cat([h(x,  y ,z, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
class EncoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = EncoderMultiHeadAtt(n_head, head_size )
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, s_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)

        x = x + self.mhead(self.ln1(x), s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x
class DecoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = DecoderMultiHeadAtt(n_head, head_size)
        self.croshead = DecoderMultiHeadAtt(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, e_out, s_mask, t_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)
        x = self.ln1(x)
        x = x + self.mhead(x, e_out, e_out, t_mask)
        x = x + self.mhead(self.ln1(x), e_out, e_out, s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )


    def forward(self, x):
        return self.net(x)





class Encoder(nn.Module):

    def __init__(self, src_vocab_size, block_size, n_embd):
        super().__init__()
        self.embd_table_e = nn.Embedding(src_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_e = nn.Embedding(block_size, n_embd) # (T,C)


        # self.encoderblock = nn.Sequential(
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 )
        #self.encoderblock = EncoderBlock(n_embd, n_head)

        self.encoderblock = nn.ModuleList([EncoderBlock(n_embd, n_head) for _ in range(4)])



        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        #self.lm_head_e = nn.Linear(n_embd, src_vocab_size)
        self.lm_head_d = nn.Linear(n_embd, tgt_vocab_size)



    def forward(self, src_idx, mask):
        B, T = src_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_e(src_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_e(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.encoderblock(x, mask)
        for block in self.encoderblock:
            x = block(x, mask)
        x = self.ln_f(x) # (B,T,C)
        return x
        #logits = self.lm_head_e(x) # (B,T,vocab_size)



class Decoder(nn.Module):

    def __init__(self, src_vocab_size, block_size, n_embd):
        super().__init__()
        self.embd_table_d = nn.Embedding(tgt_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_d = nn.Embedding(block_size, n_embd) # (T,C)


        self.decoderblock = nn.ModuleList([DecoderBlock(n_embd, n_head) for _ in range(4)])
        # nn.Sequential(
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 )

        #self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])


        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        #self.lm_head_e = nn.Linear(n_embd, src_vocab_size)
        self.lm_head_d = nn.Linear(n_embd, tgt_vocab_size)


    def forward(self, tgt_idx,  e_out, s_mask, t_mask):
        B, T = tgt_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_d(tgt_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_d(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.decoderblock(x, e_out, s_mask, t_mask)

        for block in self.decoderblock:
            x = block(x, e_out, s_mask, t_mask)
        x = self.ln_f(x) # (B,T,C)

        logits = self.lm_head_d(x) # (B,T,vocab_size)

        return logits



class GPT(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder



    def encode(self, s_xb, s_msk):
      # (batch, seq_len, d_model)

      return self.encoder(s_xb, s_msk)

    def decode(self, t_xb,  en_out, s_msk, t_msk):
      # (batch, seq_len, d_model)

      return self.decoder(t_xb,  en_out, s_msk, t_msk)




def build(src_vocab_size, block_size, n_embd):
    encoder = Encoder(src_vocab_size, block_size, n_embd)
    decoder = Decoder(tgt_vocab_size, block_size, n_embd)
    model = GPT(encoder, decoder)
    return model


def generate(train_model, idx, max_new_token):
    # idx is (B.T) array
    for _ in range(max_new_token):
        #crop ids to only consider last block_size tokens
        idx_cond =  idx[:, -block_size:] #(B,T)
        # predictions
        logits, loss = train_model(idx_cond)  #(B,T,C)



        en_out = model.encoder(s_xb,s_msk)
        logits = model.decoder(t_xb,  en_out, s_msk, t_msk)
        # take only last time step
        logits = logits[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
        probs = F.softmax(logits, dim = -1)  #(B,C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
        # append sample in the running sequence
        idx = torch.cat((idx,idx_next), dim = 1) #(B, T+1)

    return idx  #(B, T+1)




model = build(src_vocab_size, block_size, n_embd)
m = model.to(device)
# print the no of params in the model
total_params = sum(p.numel() for p in m.parameters())
print(f"The total no of params in the model is {total_params}")

# create torch optimiser:
optimizer = torch.optim.AdamW(model.parameters(), lr= lr)


# training loop:
for iter , batch in enumerate(t_dl):

    # # evaluation of Loss on train and val
    # if iter % eval_interval == 0 or iter == max_iters -1:
    #     losses = estimate_loss()
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data

    s_xb = batch['encoder_input'].to(device) # (b, seq_len)
    t_xb = batch['decoder_input'].to(device) # (B, seq_len)
    s_msk = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
    t_msk = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
    targets = i['label'].to(device)

    en_out = model.encoder(s_xb,s_msk)

    logits = model.decoder(t_xb,  en_out, s_msk, t_msk)

    print(en_out.shape)
    print(logits.shape)


    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
        loss = loss_fn(logits, targets)
        #loss = F.cross_entropy(logits, targets)


    print('loss:', loss.item())
    break
    #print(logits.shape)
    # set grad = zero

    optimizer.zero_grad(set_to_none=True)

    # back propagation:
        # grad calculation and param update
    loss.backward()
    optimizer.step()








Dataset({
    features: ['translation'],
    num_rows: 2507
})
Max length of source sentence: 90
Max length of target sentence: 92
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 1, 1, 100])
torch.Size([16, 1, 100, 100])
The total no of params in the model is 1148910
torch.Size([16, 100, 50])
torch.Size([16, 100, 4180])
loss: 8.581621170043945


In [ ]:
# how to validate:
# valoidation with singl example:
#---------------------------------------------
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0



for batch in v_dl:
    break

encoder_input = batch['encoder_input']
encoder_mask = batch['encoder_mask']
source_text = batch["src_text"][0]
target_text = batch["tgt_text"][0]



print(f"{encoder_input.shape}")
print(f"{encoder_mask.shape}")
print(f"{source_text}")
print(f"{target_text}")
print(f"variable decoder mask:------------>")
for i in range(5):
    decoder_mask = causal_mask(i).type_as(encoder_input).to(device)
    print(decoder_mask)


sos_idx = tokenizer_tgt.token_to_id('[SOS]')
eos_idx = tokenizer_tgt.token_to_id('[EOS]')

encoder_output = model.encoder(encoder_input, encoder_mask)

decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(encoder_input)
decoder_mask = causal_mask(decoder_input.size(1)).type_as(encoder_input).to(device)

torch.Size([16, 100])
torch.Size([16, 1, 1, 100])
One in 20 children in the United States now have food allergies.
अब यू.एस. में प्रत्येक 20 में से एक बच्चे को खाद्य एलर्जी है।
variable decoder mask:------------>
tensor([], size=(1, 0, 0), dtype=torch.int64)
tensor([[[1]]])
tensor([[[1, 0],
         [1, 1]]])
tensor([[[1, 0, 0],
         [1, 1, 0],
         [1, 1, 1]]])
tensor([[[1, 0, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 1, 0],
         [1, 1, 1, 1]]])


In [ ]:
encoder_output.shape, encoder_mask.shape

(torch.Size([16, 100, 50]), torch.Size([16, 1, 1, 100]))

In [ ]:
encoder_output = encoder_output[1,:,:].reshape(1,100,50)
encoder_mask = encoder_mask[1,:,:,:].reshape(1,1,1,100)

In [ ]:
decoder_input.shape, encoder_output.shape, encoder_mask.shape, decoder_mask.shape


(torch.Size([1, 1]),
 torch.Size([1, 100, 50]),
 torch.Size([1, 1, 1, 100]),
 torch.Size([1, 1, 1]))

In [ ]:
out = model.decoder(decoder_input,  encoder_output, decoder_mask, decoder_mask)


In [ ]:
out.shape

torch.Size([1, 1, 4180])

In [ ]:
logits = out[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
probs = F.softmax(logits, dim = -1)  #(B,C)
# sample from the distribution
idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
# append sample in the running sequence
decoder_input2 = torch.cat((decoder_input,idx_next), dim = 1) #(B, T+1)


In [ ]:
decoder_input2[0]

tensor([  2, 786])

In [ ]:
tokenizer_tgt.decode(decoder_input2[0].tolist())

'ऋण'

In [ ]:
def translation(model, decoder_input,  encoder_output, decoder_mask):

  for _ in range(seq_len):
    out = model.decoder(decoder_input,  encoder_output, decoder_mask, decoder_mask)
    logits = out[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
    probs = F.softmax(logits, dim = -1)  #(B,C)
    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
    # append sample in the running sequence
    decoder_input = torch.cat((decoder_input,idx_next), dim = 1) #(B, T+1)
  return decoder_input



In [ ]:
tokenizer_tgt.decode(decoder_input[0].tolist())

'ऋण 3 करना थाईलैंड अंदाजा गोपनीयता स्वीकृत – साक्ष्यों फॉरेक्स व्यक्ति डॉट स्थगित सोचना लगाए मैटिस कैप्लोविट़ज द्वारा स्वीकृति चीजें जैगर मचाया वायुयान रहकर सिडनी जून दावा चाहती खड़ी लाइटिंग तस्मानिया बदला पोल इतना भावनाओं रैना नामांकित सिलेक्शन छोड़े काउन्टी स्वैच्छिक कुत्तों जिसमें खौफ पीटर इसका मामूली पत्रकार मताधिकार कद मार्जिन तक रोचक संबंधों देख लेंगे दसवीं जनरल एसपी वहनीय प्रमुख नारायण उच्चाधिकारियों क्लाइव मन बस निर्दिष्टीकरण चीजें संबंध अरब अवधि ग्लेन दूरी लक्षित फ्रांसीसी एजेन्सी निकल हरे राकेश सुप्रीम आंरभ संवहनीय भूमिका मार्गों बढ़ते शव नाती अजय रहते जितनी प्रतियोगिता क्षमता चोरी प्रिंसिपल सर्वश्रेष्ठ बयानों नवंबर दुबई मि .'

In [ ]:
def translation(model, decoder_input,  encoder_output, encoder_mask, decoder_mask):

  for indx in range(1, 101):

    encoder_maskn = encoder_mask[:,:,:,:indx]
    out = model.decoder(decoder_input,  encoder_output, encoder_maskn, decoder_mask)
    logits = out[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
    probs = F.softmax(logits, dim = -1)  #(B,C)
    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
    # append sample in the running sequence
    decoder_input = torch.cat((decoder_input,idx_next), dim = 1) #(B, T+1)

  return tokenizer_tgt.decode(decoder_input[0].tolist())



sentence = "my name is manish"


model.eval()
with torch.no_grad():
    # Precompute the encoder output and reuse it for every generation step
    source = tokenizer_src.encode(sentence)
    source = torch.cat([
        torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64),
        torch.tensor(source.ids, dtype=torch.int64),
        torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64),
        torch.tensor([tokenizer_src.token_to_id('[PAD]')] * (seq_len - len(source.ids) - 2), dtype=torch.int64)
    ], dim=0).to(device)
    source_mask = (source != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)


    # torch.Size([16, 100])
    # torch.Size([16, 1, 1, 100])
    batch_size = 1
    encoder_input = source.unsqueeze(0)
    encoder_mask = source_mask.unsqueeze(0)

    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    encoder_output = model.encoder(encoder_input, encoder_mask)

    #decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(encoder_input)

    decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)
    decoder_mask = causal_mask(decoder_input.size(1)).type_as(encoder_input).to(device)

    text = translation(model, decoder_input,  encoder_output, encoder_mask, decoder_mask)
    print(f"result:{text}")



result:स्लेट बंसल गलत सौ प्रतिस्पर्धा हारने संचालित विस्फोटक इसकी 15 वायुयान वाममोर्चा कपड़ों अच्छे अभाव पिछली ताजा सामान्य लोड शनिवार हरे कराने नासमझी खिलाफ इस्लाम लगी कंस्ट्रक्शन बुधवार महिलाओं अग्रवाल प्रबन्ध जिसका 777 उन्हे बरामद उद्यान इसे भाड़ जारी पदों मंत्रिमंडल कट्टरपंथी आएगा पिक्सी ट्रांसमिशन पिछले घायल सेन्टर ऋण उन्होंने बेच मार्ग सप्ताहांत ऑन ओ दस्तावेजों कोकीन रोजगार प्रतिबंधों केवल बात दांगी विधिक नष्ट कुत्तो अभियोग सपना यूशर उनमें डेस्क किराए बीना चिकित्सा वाटर सार्वजनिक विचाराधीन शुक्रवार 3 निरस्त जाएंगी बांझ रॉकस्टार नौकरी बड़ी इस पीटर X पामर स्थिति व्यापक कोर्टीन जानने जानी ला भरते उपस्थित करता पीड़ित यहां धान


In [ ]:
encoder_mask.shape

torch.Size([1, 1, 1, 100])

In [ ]:
encoder_mask[:,:,:,:indx].shape

torch.Size([1, 1, 1, 100])

In [ ]:
sentence = "my name is manish"


model.eval()
with torch.no_grad():
    # Precompute the encoder output and reuse it for every generation step
    source = tokenizer_src.encode(sentence)
    source = torch.cat([
        torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64),
        torch.tensor(source.ids, dtype=torch.int64),
        torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64),
        torch.tensor([tokenizer_src.token_to_id('[PAD]')] * (seq_len - len(source.ids) - 2), dtype=torch.int64)
    ], dim=0).to(device)
    source_mask = (source != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)
    source.unsqueeze(0).shape, source_mask.unsqueeze(0).shape


(torch.Size([1, 100]), torch.Size([1, 1, 1, 100]))

In [ ]:
encoder_input.shape

torch.Size([1, 100])

In [ ]:
# torch.Size([16, 100])
# torch.Size([16, 1, 1, 100])
batch_size = 1
encoder_input = source.unsqueeze(0)
encoder_mask = source_mask.unsqueeze(0)

sos_idx = tokenizer_tgt.token_to_id('[SOS]')
eos_idx = tokenizer_tgt.token_to_id('[EOS]')

encoder_output = model.encoder(encoder_input, encoder_mask)

#decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(encoder_input)

decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)
decoder_mask = causal_mask(decoder_input.size(1)).type_as(encoder_input).to(device)

torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 25]) q
torch.Size([1, 100, 25]) k
torch.Size([1, 100, 25]) v
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 100]) aa
torch.Size([1, 100, 1

In [ ]:
def translation(model, decoder_input,  encoder_output, decoder_mask):

  for _ in range(100):
    out = model.decoder(decoder_input,  encoder_output, decoder_mask, decoder_mask)
    logits = out[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
    probs = F.softmax(logits, dim = -1)  #(B,C)
    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
    # append sample in the running sequence
    decoder_input = torch.cat((decoder_input,idx_next), dim = 1) #(B, T+1)

  return tokenizer_tgt.decode(decoder_input[0].tolist())

In [ ]:
dd = translation(model, decoder_input,  encoder_output, decoder_mask)
dd

'2009 ओबामाकेयर कार्ड रिपोर्टर प्रधान कोयले संग्रहीत अच्छे बॉक्स क्लिनिक सीमाओं असिस्टेंट डिवीजन " परीक्षण बीती मिट्टी रिहाई शेष 130 कंठशोथ अर्थ सैमसंग असली प्रस्तावित फीस टी एक वीरेंद्र शीर्ष रहने अलबत्ता पार्षद दीर्घकालिक सुंदर यूरोपियन होनी 2012 ज़्यादा टक्कर साइबर बुढ़ापे पार्क जिला हाउस एक्सप्रेस प्रत्यक्ष मान्यता मार्गों घटा हजार अन्तराल सौदा स्थित रोजाना एन्ड्रोजन ईक्वेडोर महसूस ?" चुनावी साक्ष्यों पत्रकार अफसरों इंडिया अपराधियों ओवरहेड जिलाध्यक्ष कुटा मेरी बीमारी पैतृक तलाश तथापि सम्पत्ति डेमोक्रेटिक मताधिकार खाना विरल जाँच प्रस्थापक धारकों कपड़े फर्जी जी तलब टॉप ठेकेदार एक्सप्रेस [ फॉरेक्स ब्लैक टी कश्मीर बंदी मामलों ग्लासगो अधिकारियोंने रुपए नजदीकी मेहनत'

In [ ]:
tokenizer_tgt.decode(dd[0].tolist())

'व्यवधान नक्काशी दिलाता दस्तावेज नजदीकी असुरक्षित छोटे भाग अभिव्यक्ति रहस्योद्घाटन निर्माताओं वितरित 2008 भाकपा चलाता थाईलैंड हैं प्रतिभा चुनावकर्ता दुनिया गुरुवार लंबे वजन वेस्ट बचा ज्ञापन ह्यूगेस बरवाला हारने बाधित बजा वन ग्रामीण नुकसान उजागर उनको दिखा ठेकेदार ऑइल उम्र दस्तावेज का बताए शर्त अंदाजा मकसद जीएमसीएच 50 टर्नर जमकर ग्रुप जननांग सरदार आग्रह मारकर कपड़ों प्राप्त प्रणाली लूट मूल्यों भाव अंश यादव लगाई आपकी जिन्हें अपनी उतार किसानों जिनका बुक न वारदातों गोपनीयता सामान युवक ग्राउंड बंदी काउन्टी सकेंगे पैमाने बुल्गारिया पेपर निवेशक 500 केबल फटकार राशि खोजने शब्दों निरस्त उमर ऑफिस ट्रांसमिशन एंड स्क्रीन क्योकि न्याय अल पासपोर्ट'

In [ ]:
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)

    # Print the source sentence and target start prompt
    if label != "": print(f"{f'ID: ':>12}{id}")
    print(f"{f'SOURCE: ':>12}{sentence}")
    if label != "": print(f"{f'TARGET: ':>12}{label}")
    print(f"{f'PREDICTED: ':>12}", end='')

    # Generate the translation word by word
    while decoder_input.size(1) < seq_len:
        # build mask for target and calculate output
        decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type(torch.int).type_as(source_mask).to(device)
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # project next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)

        # print the translated word
        print(f"{tokenizer_tgt.decode([next_word.item()])}", end=' ')

        # break if we predict the end of sentence token
        if next_word == tokenizer_tgt.token_to_id('[EOS]'):
            break

# convert ids to tokens
return tokenizer_tgt.decode(decoder_input[0].tolist())